# Database | Mini Project 2

### Imports

In [5]:
# utillities
import pandas as pd
import json
from timeit import timeit
from pprint import pprint

# database drivers
from neo4j import GraphDatabase
from pymongo import MongoClient

## Neo4j

### Get name of persons who acted in a movie in 2006

In [ ]:
def neo4j_query_1():
    with driver.session() as session:
        m ='''MATCH(y:Year)-[r:RELEASED]-(m:Movie)-[a:ACTED]-(p:Person)
        WHERE y.year = "2006"
        return p.name'''
        r = session.run(m)

### Get amount of persons that acted in a movie directed by David Yates

In [ ]:
def neo4j_query_2():
    with driver.session() as session:
        m ='''MATCH (d:Person)-[:DIRECTED]-(:Movie)-[:ACTED]-(a:Person) 
        WHERE d.name = "David Yates"
        RETURN count(distinct a)'''
        r = session.run(m)

### Get genres Christian Bale appeared in

In [ ]:
def neo4j_query_3():
    with driver.session() as session:
        m ='''MATCH (:Person {name:"Christian Bale"})-[:ACTED]-(:Movie)-[:GENRE]-(g:Genre) 
        RETURN count(g), g.name'''
        r = session.run(m)

### Performance Time

In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "1234"))

print("neo4j query 1:", timeit(neo4j_query_1, number=5000))
print("neo4j query 2:", timeit(neo4j_query_2, number=5000))
print("neo4j query 3:", timeit(neo4j_query_3, number=5000))

driver.close()

## Mongo DB

### Get name of persons who acted in a movie in 2006

In [1]:
def mongo_query_1():
    response = mongo_col.aggregate([
        { '$match': { 'Year': 2006 } },
        { '$project': { 'a': '$Actors' } },
        { '$unwind': '$a' },
        {
            '$group': {
                '_id': None, 
                'res': {
                    '$addToSet': '$a'
                }
            }
        }
    ])

    response.next()['res']

### Get amount of persons that acted in a movie directed by David Yates

In [2]:
def mongo_query_2():
    response = mongo_col.aggregate([ 
        { '$match' : { 'Director':'David Yates' } }, 
        {'$group': {
            '_id': 0,
            "actors": { '$push': '$Actors' }
        }},
        { '$project': {
            '_id': 0, 
            'unique_actors': { '$size' : { 
                '$reduce': {
                    'input': '$actors',
                    'initialValue': [],
                    'in': { '$setUnion': [ '$$value', '$$this' ] }
                }
            }} 
        }}
    ]);

    return response.next()['unique_actors']

### Get genres Christian Bale appeared in

In [3]:
def mongo_query_3():
    response = mongo_col.aggregate([ 
        { '$match' : { 'Actors': { '$elemMatch': { '$eq': 'Christian Bale' } }}}, 
        {'$group': {
            '_id': 0,
            "genres": { '$push': '$Genre' }
        }},
        { '$project': {
            '_id': 0, 
            'unique_genres': { 
                '$reduce': {
                    'input': '$genres',
                    'initialValue': [],
                    'in': { '$setUnion': [ '$$value', '$$this' ] }
                }
            }
        }}
    ]);

    return response.next()['unique_genres']

### Performance Time

In [6]:
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["mini_project"]
mongo_col = mongo_db["main"]

print("mongo query 1:", timeit(mongo_query_1, number=5000))
print("mongo query 2:", timeit(mongo_query_2, number=5000))
print("mongo query 3:", timeit(mongo_query_3, number=5000))

mongo query 1: 27.429045424999913
mongo query 2: 26.790858463000063
mongo query 3: 23.18313161100002
